# Final Project, COMS 4995_005, Deep Learning

## Team Information

Team Member1 (Name,UNI): Richard Kennedy, rk2860

Python Version: 2
Utilizing Pytorch

In [1]:
import torch
import cPickle
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchtext.vocab as Vocab

import torchvision.datasets as dset
import torchvision.transforms as transforms

import PIL

In [25]:
cap = dset.CocoCaptions(root = './train2014',
                        annFile = 'annotations/captions_train2014.json',
                        transform= transforms.Compose([transforms.Resize([224, 224]), transforms.ToTensor()]))

res = models.resnet50(pretrained=True)
x = Vocab.GloVe(name='6B', dim= 50)



loading annotations into memory...
Done (t=5.76s)
creating index...
index created!


In [46]:
a = x.vectors[x.stoi['man']]
print a.size()
b = torch.FloatTensor(1998).zero_()
paddedVec = torch.cat((a, b), 0)



torch.Size([50])

-0.0944
 0.4301
-0.1722
   ⋮   
 0.0000
 0.0000
 0.0000
[torch.FloatTensor of size 2048]



In [34]:
print "HELLO"
img, target = cap[0]
words = ['StopLSTM'] + target[0].split(' ') + ['EndLSTM']
print words


print wordVec(Vocab[words[1]])

HELLO
['StopLSTM', u'People', u'shopping', u'in', u'an', u'open', u'market', u'for', u'vegetables.', 'EndLSTM']


NameError: name 'wordVec' is not defined

## Part 1

In [76]:
class LSTM(nn.Module):
    def __init__(self, embed_size, hidden_size, res50):
        super(LSTM, self).__init__()
        
        self.Esize = embed_size
        self.Hsize = hidden_size
        self.Wix = nn.Linear(embed_size, hidden_size)
        self.Wim = nn.Linear(hidden_size, hidden_size)
        self.Wfx = nn.Linear(embed_size, hidden_size)
        self.Wfm = nn.Linear(hidden_size, hidden_size)
        self.Wox = nn.Linear(embed_size, hidden_size)
        self.Wom = nn.Linear(hidden_size, hidden_size)
        self.Wcx = nn.Linear(embed_size, hidden_size)
        self.Wcm = nn.Linear(hidden_size, hidden_size)
        
        #res50 = torchvision.models.resnet50(pretrained=True)
        self.res50NoGrad = torch.nn.Sequential(*list(res50.children())[:-2])
        #Every layer besides the last layer does not need to be trained
        for param in self.res50NoGrad.parameters():
            param.requires_grad = False
        
        #set embed_size to dimensions of self.res50Last
        #output should be second to last layer
        self.res50Last = torch.nn.Sequential(list(res50.children())[-2])
        
        
        #self.glove = Vocab.GloVe(name='840B', dim= embed_size)
        
        
        
        

    def forward(self, x, m = None, c = None, CNN = False ):
        
        if not CNN:
            #glove requires x to be one hot word encoding?
            #x = self.glove(x)
            i = F.sigmoid(self.Wix(x)+self.Wim(m))
            f = F.sigmoid(self.Wfx(x)+self.Wfm(m))
            o = F.sigmoid(self.Wox(x)+self.Wom(m))
            h = F.tanh(self.Wcx(x) + self.Wcm(m))
            newC = f*c + i*h
            newM = o*newC
            #prediction dimension should line up with vocab_size
            #fine tune hidden dimension to make that work
            
            #log_softmax might not be necessary -> due to backprop
            
            #calculations: hidden dimensions = 50, embed_size = 2048, vocab_size not used
            prediction = F.log_softmax(newM,)
            
            return prediction, newC, newM
        
        else:
            x = self.res50NoGrad(x)
            x = self.res50Last(x)
            x = x.resize(1, self.Esize)
            i = F.sigmoid(self.Wix(x))
            f = F.sigmoid(self.Wfx(x))
            o = F.sigmoid(self.Wox(x))
            h = F.tanh(self.Wcx(x))
            c = i*h
            m = o*c
            
            return None, c, m

net = LSTM(2048, 50, res)
img, target = cap[0]
img = img.unsqueeze(0)
junk, c0, m0 =  net.forward(Variable(img), CNN = True)

a = x.vectors[x.stoi['man']]
b = torch.FloatTensor(1998).zero_()
paddedVec = torch.cat((a, b), 0)

pred, c0, m0 = net.forward(Variable(paddedVec), m = m0, c = c0)
pred = pred.unsqueeze(0)


 0.9528
-0.2061
 0.5562
-0.4632
 0.7335
 0.0291
-0.1937
-0.0901
-0.2296
-0.1906
-0.3486
-1.0231
 0.7430
-0.5489
 0.8848
-0.1405
 0.0040
 0.5845
 0.1077
-0.4466
-0.4320
 0.9868
 0.7829
 0.5151
 0.8579
-1.7713
-0.8826
-0.5973
 0.0849
-0.4811
 3.9678
 0.8893
-0.2706
-0.4409
-0.2621
 0.0856
 0.0221
-0.5838
 0.1091
 0.7797
-0.9545
 0.4048
 0.8941
 0.6525
 0.3986
 0.2088
-1.3281
-0.1088
-0.2282
-0.4630
[torch.FloatTensor of size 50]



/Users/Richard/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:50: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


## Part 2

In [101]:
##Train


## was lazy and used cat and dog as start and stop word. 
## Definetely going to have to change that
net = LSTM(2048, 50, res)
import torch.optim as optim

criterion = nn.NLLLoss()
optimizer = optim.SGD(filter(lambda p: p.requires_grad,net.parameters()), lr=0.003, momentum=.7)
for i in range(0, 100):
    img, target = cap[i]
    words = ['cat'] + target[0].split(' ') + ['dog']
    length = len(words)
    img = img.unsqueeze(0)
    junk, c0, m0 =  net.forward(Variable(img), CNN = True)
    fullPred = None
    fullAnsw = None
    for j in range(0, length-1):
        #Going to need larger word corpus
        if words[j].lower() in x.stoi and words[j+1].lower() in x.stoi:
            
            a = x.vectors[x.stoi[words[j].lower()]]
            b = torch.FloatTensor(1998).zero_()
            paddedVec = torch.cat((a, b), 0)
            pred, c0, m0 = net.forward(Variable(paddedVec), m = m0, c = c0)


            answer = x.vectors[x.stoi[words[j+1].lower()]].unsqueeze(0)
            
            criterion(pred, Variable(answer.type('torch.LongTensor')))
            loss.backward()
            optimizer.step()
    
            print loss.data[0]
           
    
#             if j == 0:
#                 fullPred = pred
#                 fullAnsw = answer.unsqueeze(0)
#             else:
#                 answer = answer.unsqueeze(0)
#                 fullPred = torch.cat((fullPred, pred), 0)
#                 fullAnsw = torch.cat((fullAnsw, answer), 0)
                
    
    
    
    print fullPred.size()
    print fullAnsw.size()
    
    
            
    
    

/Users/Richard/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:50: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


RuntimeError: multi-target not supported at /Users/soumith/minicondabuild3/conda-bld/pytorch_1518371252923/work/torch/lib/THNN/generic/ClassNLLCriterion.c:22

## Extra Credit